# Lesson I7: Module System & Organization

**Duration**: 90-105 minutes  
**Stage**: Intermediate (Building Skills)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Organize code using modules and the `mod` keyword
2. Control visibility with `pub` and understand privacy rules
3. Use `use` statements to bring items into scope
4. Structure larger projects with multiple files and directories
5. Work with external crates and dependencies

---

## 📋 Prerequisite Review

**Quick Check**: From our previous lessons:

1. How do you define a trait with default implementations?
2. What's the difference between static and dynamic dispatch?
3. How do you use trait bounds in generic functions?
4. What are associated types in traits?

**Answers**: 1) `trait Name { fn method(&self) { /* default */ } }`, 2) Static is compile-time, dynamic uses trait objects, 3) `fn func<T: Trait>(param: T)`, 4) Types associated with trait implementations

---

## 🧠 Key Concepts

### Module System Basics

**Modules** organize code:
- **Namespacing**: Prevent name conflicts
- **Privacy**: Control what's accessible
- **Organization**: Group related functionality
- **Compilation**: Separate compilation units

### Privacy Rules

- **Default Private**: Items are private by default
- **pub Keyword**: Makes items public
- **Parent Access**: Child modules can access parent's private items
- **Sibling Access**: Siblings can access each other's public items

---

## 🔬 Live Code Exploration

### Basic Module Definition

In [ ]:
// Basic module definition and usage

mod math_utils {
    // Private function (default)
    fn internal_calculation(x: i32) -> i32 {
        x * x + 2 * x + 1
    }
    
    // Public function
    pub fn square(x: i32) -> i32 {
        x * x
    }
    
    pub fn cube(x: i32) -> i32 {
        x * x * x
    }
    
    pub fn perfect_square(x: i32) -> i32 {
        internal_calculation(x) // Can access private function within module
    }
    
    // Public constant
    pub const PI: f64 = 3.14159265359;
    
    // Private constant
    const INTERNAL_FACTOR: i32 = 42;
    
    pub fn get_factor() -> i32 {
        INTERNAL_FACTOR
    }
}

mod string_utils {
    pub fn reverse_string(s: &str) -> String {
        s.chars().rev().collect()
    }
    
    pub fn capitalize_first(s: &str) -> String {
        let mut chars = s.chars();
        match chars.next() {
            None => String::new(),
            Some(first) => first.to_uppercase().collect::<String>() + chars.as_str(),
        }
    }
    
    pub fn word_count(s: &str) -> usize {
        s.split_whitespace().count()
    }
}

fn basic_modules_demo() {
    println!("=== Basic Modules Demo ===");
    
    // Using functions from math_utils module
    let num = 5;
    println!("Number: {}", num);
    println!("Square: {}", math_utils::square(num));
    println!("Cube: {}", math_utils::cube(num));
    println!("Perfect square: {}", math_utils::perfect_square(num));
    println!("PI constant: {}", math_utils::PI);
    println!("Internal factor: {}", math_utils::get_factor());
    
    // This would cause a compile error (private function):
    // math_utils::internal_calculation(5);
    
    println!();
    
    // Using functions from string_utils module
    let text = "hello world";
    println!("Original: '{}'", text);
    println!("Reversed: '{}'", string_utils::reverse_string(text));
    println!("Capitalized: '{}'", string_utils::capitalize_first(text));
    println!("Word count: {}", string_utils::word_count(text));
}

basic_modules_demo();

### Nested Modules and Use Statements

In [ ]:
// Nested modules and use statements

mod graphics {
    pub mod shapes {
        pub struct Circle {
            pub radius: f64,
        }
        
        impl Circle {
            pub fn new(radius: f64) -> Self {
                Circle { radius }
            }
            
            pub fn area(&self) -> f64 {
                std::f64::consts::PI * self.radius * self.radius
            }
        }
        
        pub struct Rectangle {
            pub width: f64,
            pub height: f64,
        }
        
        impl Rectangle {
            pub fn new(width: f64, height: f64) -> Self {
                Rectangle { width, height }
            }
            
            pub fn area(&self) -> f64 {
                self.width * self.height
            }
        }
    }
    
    pub mod colors {
        #[derive(Debug, Clone)]
        pub struct Color {
            pub r: u8,
            pub g: u8,
            pub b: u8,
        }
        
        impl Color {
            pub fn new(r: u8, g: u8, b: u8) -> Self {
                Color { r, g, b }
            }
            
            pub fn red() -> Self {
                Color::new(255, 0, 0)
            }
            
            pub fn green() -> Self {
                Color::new(0, 255, 0)
            }
            
            pub fn blue() -> Self {
                Color::new(0, 0, 255)
            }
        }
    }
    
    pub mod drawing {
        use super::shapes::{Circle, Rectangle};
        use super::colors::Color;
        
        pub struct Canvas {
            width: u32,
            height: u32,
        }
        
        impl Canvas {
            pub fn new(width: u32, height: u32) -> Self {
                Canvas { width, height }
            }
            
            pub fn draw_circle(&self, circle: &Circle, color: &Color) {
                println!("Drawing circle (radius: {:.1}) in color {:?} on {}x{} canvas", 
                        circle.radius, color, self.width, self.height);
            }
            
            pub fn draw_rectangle(&self, rect: &Rectangle, color: &Color) {
                println!("Drawing rectangle ({}x{}) in color {:?} on {}x{} canvas", 
                        rect.width, rect.height, color, self.width, self.height);
            }
        }
    }
}

// Different ways to use modules
use graphics::shapes::Circle;
use graphics::shapes::Rectangle as Rect; // Alias
use graphics::colors::*; // Glob import (use sparingly)
use graphics::drawing::Canvas;

fn nested_modules_demo() {
    println!("\n=== Nested Modules and Use Statements ===");
    
    // Create shapes using imported types
    let circle = Circle::new(5.0);
    let rectangle = Rect::new(10.0, 8.0);
    
    println!("Circle area: {:.2}", circle.area());
    println!("Rectangle area: {:.2}", rectangle.area());
    
    // Create colors using glob import
    let red = Color::red();
    let custom_color = Color::new(128, 64, 192);
    
    // Create canvas and draw
    let canvas = Canvas::new(800, 600);
    
    canvas.draw_circle(&circle, &red);
    canvas.draw_rectangle(&rectangle, &custom_color);
    
    // Using full path (without use statement)
    let blue_circle = graphics::shapes::Circle::new(3.0);
    let blue_color = graphics::colors::Color::blue();
    canvas.draw_circle(&blue_circle, &blue_color);
}

nested_modules_demo();

### Advanced Module Patterns

In [ ]:
// Advanced module patterns and organization

mod database {
    // Private module - only accessible within database module
    mod connection {
        pub struct Connection {
            url: String,
            connected: bool,
        }
        
        impl Connection {
            pub fn new(url: String) -> Self {
                println!("Creating connection to: {}", url);
                Connection {
                    url,
                    connected: false,
                }
            }
            
            pub fn connect(&mut self) -> Result<(), String> {
                if self.connected {
                    return Err("Already connected".to_string());
                }
                
                println!("Connecting to database...");
                self.connected = true;
                Ok(())
            }
            
            pub fn is_connected(&self) -> bool {
                self.connected
            }
            
            pub fn execute_query(&self, query: &str) -> Result<Vec<String>, String> {
                if !self.connected {
                    return Err("Not connected to database".to_string());
                }
                
                println!("Executing query: {}", query);
                // Simulate query results
                Ok(vec!["row1".to_string(), "row2".to_string(), "row3".to_string()])
            }
        }
    }
    
    // Re-export from private module
    pub use connection::Connection;
    
    pub mod models {
        #[derive(Debug)]
        pub struct User {
            pub id: u32,
            pub name: String,
            pub email: String,
        }
        
        impl User {
            pub fn new(id: u32, name: String, email: String) -> Self {
                User { id, name, email }
            }
        }
        
        #[derive(Debug)]
        pub struct Post {
            pub id: u32,
            pub user_id: u32,
            pub title: String,
            pub content: String,
        }
        
        impl Post {
            pub fn new(id: u32, user_id: u32, title: String, content: String) -> Self {
                Post { id, user_id, title, content }
            }
        }
    }
    
    pub mod repository {
        use super::Connection;
        use super::models::{User, Post};
        
        pub struct UserRepository {
            connection: Connection,
        }
        
        impl UserRepository {
            pub fn new(mut connection: Connection) -> Result<Self, String> {
                connection.connect()?;
                Ok(UserRepository { connection })
            }
            
            pub fn find_user(&self, id: u32) -> Result<User, String> {
                let query = format!("SELECT * FROM users WHERE id = {}", id);
                let _results = self.connection.execute_query(&query)?;
                
                // Simulate user creation from query results
                Ok(User::new(
                    id,
                    format!("User{}", id),
                    format!("user{}@example.com", id)
                ))
            }
            
            pub fn create_user(&self, name: String, email: String) -> Result<User, String> {
                let query = format!("INSERT INTO users (name, email) VALUES ('{}', '{}')", name, email);
                self.connection.execute_query(&query)?;
                
                // Simulate returning created user
                Ok(User::new(1, name, email))
            }
        }
        
        pub struct PostRepository {
            connection: Connection,
        }
        
        impl PostRepository {
            pub fn new(mut connection: Connection) -> Result<Self, String> {
                connection.connect()?;
                Ok(PostRepository { connection })
            }
            
            pub fn find_posts_by_user(&self, user_id: u32) -> Result<Vec<Post>, String> {
                let query = format!("SELECT * FROM posts WHERE user_id = {}", user_id);
                let _results = self.connection.execute_query(&query)?;
                
                // Simulate posts creation
                Ok(vec![
                    Post::new(1, user_id, "First Post".to_string(), "Hello World!".to_string()),
                    Post::new(2, user_id, "Second Post".to_string(), "Learning Rust!".to_string()),
                ])
            }
        }
    }
    
    // Public API that hides internal complexity
    pub fn create_user_repository(database_url: &str) -> Result<repository::UserRepository, String> {
        let connection = Connection::new(database_url.to_string());
        repository::UserRepository::new(connection)
    }
    
    pub fn create_post_repository(database_url: &str) -> Result<repository::PostRepository, String> {
        let connection = Connection::new(database_url.to_string());
        repository::PostRepository::new(connection)
    }
}

// Application module that uses the database module
mod app {
    use super::database;
    
    pub fn run_app() -> Result<(), String> {
        println!("Starting application...");
        
        let database_url = "postgresql://localhost:5432/myapp";
        
        // Create repositories
        let user_repo = database::create_user_repository(database_url)?;
        let post_repo = database::create_post_repository(database_url)?;
        
        // Use repositories
        let user = user_repo.find_user(1)?;
        println!("Found user: {:?}", user);
        
        let posts = post_repo.find_posts_by_user(user.id)?;
        println!("User's posts:");
        for post in posts {
            println!("  - {}: {}", post.title, post.content);
        }
        
        // Create new user
        let new_user = user_repo.create_user(
            "Alice".to_string(), 
            "alice@example.com".to_string()
        )?;
        println!("Created new user: {:?}", new_user);
        
        Ok(())
    }
}

fn advanced_modules_demo() {
    println!("\n=== Advanced Module Patterns ===");
    
    match app::run_app() {
        Ok(()) => println!("Application completed successfully!"),
        Err(e) => println!("Application error: {}", e),
    }
}

advanced_modules_demo();

---

## 🎯 Guided Practice

### Exercise 1: Library Management System

Create a modular library management system with proper organization.

In [ ]:
// TODO: Complete the library management system with proper module organization

mod library {
    pub mod books {
        #[derive(Debug, Clone)]
        pub struct Book {
            pub id: u32,
            pub title: String,
            pub author: String,
            pub isbn: String,
            pub available: bool,
        }
        
        impl Book {
            pub fn new(id: u32, title: String, author: String, isbn: String) -> Self {
                Book {
                    id,
                    title,
                    author,
                    isbn,
                    available: true,
                }
            }
            
            pub fn borrow_book(&mut self) -> Result<(), String> {
                if self.available {
                    self.available = false;
                    Ok(())
                } else {
                    Err(format!("Book '{}' is not available", self.title))
                }
            }
            
            pub fn return_book(&mut self) {
                self.available = true;
            }
        }
    }
    
    pub mod members {
        #[derive(Debug, Clone)]
        pub struct Member {
            pub id: u32,
            pub name: String,
            pub email: String,
            borrowed_books: Vec<u32>, // Book IDs
        }
        
        impl Member {
            pub fn new(id: u32, name: String, email: String) -> Self {
                Member {
                    id,
                    name,
                    email,
                    borrowed_books: Vec::new(),
                }
            }
            
            pub fn borrow_book(&mut self, book_id: u32) -> Result<(), String> {
                if self.borrowed_books.len() >= 5 {
                    return Err("Cannot borrow more than 5 books".to_string());
                }
                
                if self.borrowed_books.contains(&book_id) {
                    return Err("Already borrowed this book".to_string());
                }
                
                self.borrowed_books.push(book_id);
                Ok(())
            }
            
            pub fn return_book(&mut self, book_id: u32) -> Result<(), String> {
                if let Some(pos) = self.borrowed_books.iter().position(|&id| id == book_id) {
                    self.borrowed_books.remove(pos);
                    Ok(())
                } else {
                    Err("Book not borrowed by this member".to_string())
                }
            }
            
            pub fn borrowed_books(&self) -> &[u32] {
                &self.borrowed_books
            }
        }
    }
    
    pub mod catalog {
        use super::books::Book;
        use std::collections::HashMap;
        
        pub struct BookCatalog {
            books: HashMap<u32, Book>,
            next_id: u32,
        }
        
        impl BookCatalog {
            pub fn new() -> Self {
                BookCatalog {
                    books: HashMap::new(),
                    next_id: 1,
                }
            }
            
            pub fn add_book(&mut self, title: String, author: String, isbn: String) -> u32 {
                let id = self.next_id;
                let book = Book::new(id, title, author, isbn);
                self.books.insert(id, book);
                self.next_id += 1;
                id
            }
            
            pub fn find_book(&self, id: u32) -> Option<&Book> {
                self.books.get(&id)
            }
            
            pub fn find_book_mut(&mut self, id: u32) -> Option<&mut Book> {
                self.books.get_mut(&id)
            }
            
            pub fn search_by_title(&self, title: &str) -> Vec<&Book> {
                self.books
                    .values()
                    .filter(|book| book.title.to_lowercase().contains(&title.to_lowercase()))
                    .collect()
            }
            
            pub fn search_by_author(&self, author: &str) -> Vec<&Book> {
                self.books
                    .values()
                    .filter(|book| book.author.to_lowercase().contains(&author.to_lowercase()))
                    .collect()
            }
            
            pub fn available_books(&self) -> Vec<&Book> {
                self.books
                    .values()
                    .filter(|book| book.available)
                    .collect()
            }
            
            pub fn all_books(&self) -> Vec<&Book> {
                self.books.values().collect()
            }
        }
    }
    
    pub mod management {
        use super::books::Book;
        use super::members::Member;
        use super::catalog::BookCatalog;
        use std::collections::HashMap;
        
        pub struct LibrarySystem {
            catalog: BookCatalog,
            members: HashMap<u32, Member>,
            next_member_id: u32,
        }
        
        impl LibrarySystem {
            pub fn new() -> Self {
                LibrarySystem {
                    catalog: BookCatalog::new(),
                    members: HashMap::new(),
                    next_member_id: 1,
                }
            }
            
            pub fn add_book(&mut self, title: String, author: String, isbn: String) -> u32 {
                self.catalog.add_book(title, author, isbn)
            }
            
            pub fn register_member(&mut self, name: String, email: String) -> u32 {
                let id = self.next_member_id;
                let member = Member::new(id, name, email);
                self.members.insert(id, member);
                self.next_member_id += 1;
                id
            }
            
            pub fn borrow_book(&mut self, member_id: u32, book_id: u32) -> Result<(), String> {
                // Check if member exists
                let member = self.members.get_mut(&member_id)
                    .ok_or("Member not found")?;
                
                // Check if book exists and is available
                let book = self.catalog.find_book_mut(book_id)
                    .ok_or("Book not found")?;
                
                // Try to borrow the book
                book.borrow_book()?;
                member.borrow_book(book_id)?;
                
                println!("✅ {} borrowed '{}'", member.name, book.title);
                Ok(())
            }
            
            pub fn return_book(&mut self, member_id: u32, book_id: u32) -> Result<(), String> {
                // Check if member exists
                let member = self.members.get_mut(&member_id)
                    .ok_or("Member not found")?;
                
                // Check if book exists
                let book = self.catalog.find_book_mut(book_id)
                    .ok_or("Book not found")?;
                
                // Return the book
                member.return_book(book_id)?;
                book.return_book();
                
                println!("📚 {} returned '{}'", member.name, book.title);
                Ok(())
            }
            
            pub fn search_books(&self, query: &str) -> Vec<&Book> {
                let mut results = self.catalog.search_by_title(query);
                results.extend(self.catalog.search_by_author(query));
                results.sort_by_key(|book| book.id);
                results.dedup_by_key(|book| book.id);
                results
            }
            
            pub fn member_status(&self, member_id: u32) -> Result<(), String> {
                let member = self.members.get(&member_id)
                    .ok_or("Member not found")?;
                
                println!("\n👤 Member: {} ({})", member.name, member.email);
                println!("📚 Borrowed books ({}):", member.borrowed_books().len());
                
                for &book_id in member.borrowed_books() {
                    if let Some(book) = self.catalog.find_book(book_id) {
                        println!("  - {} by {}", book.title, book.author);
                    }
                }
                
                Ok(())
            }
            
            pub fn library_status(&self) {
                let all_books = self.catalog.all_books();
                let available_books = self.catalog.available_books();
                
                println!("\n📊 Library Status:");
                println!("  Total books: {}", all_books.len());
                println!("  Available books: {}", available_books.len());
                println!("  Borrowed books: {}", all_books.len() - available_books.len());
                println!("  Total members: {}", self.members.len());
            }
        }
    }
    
    // Public API
    pub use management::LibrarySystem;
    pub use books::Book;
    pub use members::Member;
}

// Import what we need
use library::LibrarySystem;

fn library_system_demo() {
    println!("\n=== Library Management System Demo ===");
    
    let mut library = LibrarySystem::new();
    
    // Add books
    let book1 = library.add_book(
        "The Rust Programming Language".to_string(),
        "Steve Klabnik".to_string(),
        "978-1593278281".to_string()
    );
    
    let book2 = library.add_book(
        "Programming Rust".to_string(),
        "Jim Blandy".to_string(),
        "978-1492052590".to_string()
    );
    
    let book3 = library.add_book(
        "Rust in Action".to_string(),
        "Tim McNamara".to_string(),
        "978-1617294556".to_string()
    );
    
    // Register members
    let member1 = library.register_member(
        "Alice Johnson".to_string(),
        "alice@example.com".to_string()
    );
    
    let member2 = library.register_member(
        "Bob Smith".to_string(),
        "bob@example.com".to_string()
    );
    
    // Show initial status
    library.library_status();
    
    // Borrow books
    println!("\n📖 Borrowing books:");
    library.borrow_book(member1, book1).unwrap();
    library.borrow_book(member1, book2).unwrap();
    library.borrow_book(member2, book3).unwrap();
    
    // Show member status
    library.member_status(member1).unwrap();
    library.member_status(member2).unwrap();
    
    // Try to borrow unavailable book
    println!("\n❌ Trying to borrow unavailable book:");
    if let Err(e) = library.borrow_book(member2, book1) {
        println!("Error: {}", e);
    }
    
    // Return a book
    println!("\n📤 Returning book:");
    library.return_book(member1, book1).unwrap();
    
    // Search books
    println!("\n🔍 Searching for 'Rust' books:");
    let search_results = library.search_books("Rust");
    for book in search_results {
        let status = if book.available { "Available" } else { "Borrowed" };
        println!("  - '{}' by {} [{}]", book.title, book.author, status);
    }
    
    // Final status
    library.library_status();
}

library_system_demo();

---

## 🧪 Active Recall Checkpoint

**Test your understanding without looking back:**

1. What's the difference between `mod` and `use`?
2. How do you make a module item public?
3. What is the purpose of `pub use`?
4. How do you access items in parent modules?
5. What's the difference between `use crate::module` and `use super::module`?
6. How do you create module aliases?
7. What are the privacy rules for modules?
8. When should you use glob imports (`use module::*`)?

**Write your answers below:**

**Your Answers:**
1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 

---

## 🤔 Reflection Prompt

Consider these questions:

1. **How do modules help organize large codebases?**
2. **What are the benefits of Rust's privacy-by-default approach?**
3. **How do you decide what should be public vs private?**
4. **What strategies help maintain clean module boundaries?**

Write your thoughts below:

**Your Reflections:**

1. 

2. 

3. 

4. 

---

## 🔮 Preview & Connections

### Coming Up Next: Testing & Documentation

In our next lesson, you'll learn about:
- Writing unit tests and integration tests
- Documentation comments and doctests
- Test organization and best practices
- Continuous integration considerations

### How This Connects
Module organization is fundamental to:
- Structuring large applications
- Creating reusable libraries
- Managing dependencies and APIs
- Enabling effective testing strategies

---

## ✅ Expected Outcomes

**Self-Assessment Checklist** - Can you:

- [ ] Create and organize modules effectively?
- [ ] Control visibility with `pub` appropriately?
- [ ] Use `use` statements to manage imports?
- [ ] Structure nested modules logically?
- [ ] Apply privacy rules correctly?
- [ ] Re-export items with `pub use`?
- [ ] Design clean module APIs?

If you checked all boxes, excellent! You've mastered Rust's module system.

---

**🎉 Excellent Work!** You now know how to organize and structure Rust code effectively using the module system!